# comercio_ext_indices.tb_referencia_isic_cuci
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/ISIC_CUCI_delta`
## 📌 Descrição do arquivo

Tabela de **classificação econômica internacional**, que relaciona **atividades econômicas (ISIC)** com **grupos de produtos (CUCI)**.

Esse arquivo **não contém valores, volumes ou operações comerciais**.  
Ele existe exclusivamente para **interpretação econômica e setorial**.

Fonte: MDIC / classificações internacionais (ISIC / CUCI)  
Tipo: Tabela de referência (dimensão / bridge)

| Coluna          | Descrição técnica    | Interpretação humana             |
| --------------- | -------------------- | -------------------------------- |
| `CO_ISIC_SECAO` | Código da seção ISIC | Macroatividade econômica         |
| `NO_ISIC_SECAO` | Nome da seção ISIC   | Descrição da atividade econômica |
| `CO_CUCI_GRUPO` | Código do grupo CUCI | Grupo internacional de produtos  |
| `NO_CUCI_GRUPO` | Nome do grupo CUCI   | Tipo de bem comercializado       |

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/isic_cuci/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_isic_cuci/"
silverTable = "silver_comercio_ext_indices.tb_referencia_isic_cuci"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_ISIC_SECAO", T.StringType(), False),
    T.StructField("NO_ISIC_SECAO", T.StringType(), False),
    T.StructField("CO_CUCI_GRUPO", T.StringType(), False),
    T.StructField("NO_CUCI_GRUPO", T.StringType(), False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_ISIC_SECAO", up("CO_ISIC_SECAO"))
    .withColumn("NO_ISIC_SECAO", F.col("NO_ISIC_SECAO").cast(T.StringType()))
    .withColumn("CO_CUCI_GRUPO", up("CO_CUCI_GRUPO"))
    .withColumn("NO_CUCI_GRUPO", F.col("NO_CUCI_GRUPO").cast(T.StringType()))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_ISIC_SECAO").isNotNull() & F.col("CO_CUCI_GRUPO").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_ISIC_SECAO","CO_CUCI_GRUPO"])

In [0]:
df_silver = df_dedup.select("CO_ISIC_SECAO","NO_ISIC_SECAO","CO_CUCI_GRUPO","NO_CUCI_GRUPO")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_referencia_isic_cuci")

merge_condition = """
  t.CO_ISIC_SECAO   = s.CO_ISIC_SECAO   AND
  t.CO_CUCI_GRUPO   = s.CO_CUCI_GRUPO
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_ISIC_SECAO": "s.NO_ISIC_SECAO",
        "t.NO_CUCI_GRUPO": "s.NO_CUCI_GRUPO",
        "t.TS_REF": "s.TS_REF",
        "t.NM_ORIGEM": "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)